In [1]:
import os, sys
dir2 = os.path.abspath('')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path: sys.path.append(dir1)

import numpy as np
from Simulation.projections_util import Objective, create_neat_proj, create_paper_objectives
from Data.data_load_util import make_dataset
from Data.data_manager import DataManager
from scipy.optimize import milp, LinearConstraint, Bounds

In [ ]:
zips_df, state_df, pos_df = make_dataset(granularity='both', remove_outliers=False, load_dir_prefix='../Data/')
data_manager = DataManager(zips_df, fields=['carbon_offset_metric_tons_per_panel', 'yearly_sunlight_kwh_kw_threshold_avg'])

num_zips = len(zips_df)

#objective weights
weights = [1, 1, np.float64(1.487431321576903), 1]

In [3]:
#prepare constants
num_panels = 2000000
energy_potential_by_zip = data_manager.normalized_df['yearly_sunlight_kwh_kw_threshold_avg'].values
carbon_offset_by_zip = data_manager.normalized_df['carbon_offset_metric_tons_per_panel'].values

#high black prop flag
black_prop_median = zips_df['black_prop'].median()
zips_df['black_prop_flag'] = 0
zips_df.loc[zips_df['black_prop'] > black_prop_median, 'black_prop_flag'] = 1
high_black_prop_flag = zips_df['black_prop_flag'].values

#high income flag
income_median = zips_df['Median_income'].median()
zips_df['income_flag'] = 0
zips_df.loc[zips_df['Median_income'] > income_median, 'income_flag'] = 1
high_income_flag = zips_df['income_flag'].values

In [4]:
#TEST RUN WITH NO EQUITY
#optimize num panels in each zip
# zip_placements = [weights[0] * energy_potential_by_zip[i]/num_panels + 
#                weights[1] * carbon_offset_by_zip[i]/num_panels for i in range(num_zips)]

# zip_placement_bounds = data_manager.combined_df['count_qualified'].values

# total_objective = -np.array(zip_placements)

# # Variable bound
# bounds = Bounds(np.array([0 for i in range(num_zips)]), np.array(zip_placement_bounds))

# # Constraints matrix and bounds
# A = np.array([
#     [1 for i in range(num_zips)] # sum of total panels is num_panels
# ])
# lb = np.array([num_panels])
# ub = np.array([num_panels])
# constraints = LinearConstraint(A, lb, ub)

# # Integer constraint
# integrality = np.array([1 for i in range(num_zips)])  # 1 = integer, 0 = continuous

# # Solve MILP
# res = milp(c=total_objective, constraints=constraints, integrality=integrality, bounds=bounds)

# # Output result
# print("Status:", res.message)
# print("Objective value:", res.fun)
# print("x =", res.x)


In [5]:
#optimize num panels in each zip
zip_placements = [weights[0] * energy_potential_by_zip[i]/num_panels + 
               weights[1] * carbon_offset_by_zip[i]/num_panels for i in range(num_zips)]

zip_placement_bounds = data_manager.combined_df['count_qualified'].values.tolist()
#aux vars for abs val when calculating equity
auxilliary_vars = [-weights[2]/num_panels, -weights[3]/num_panels]

total_objective = -np.array(zip_placements + auxilliary_vars)

# Variable bound
bounds = Bounds([0 for i in range(num_zips + 2)], zip_placement_bounds+ [np.inf, np.inf])

# Constraints matrix and bounds
A = np.array([
    [1 for i in range(num_zips)] + [0, 0], # sum of total panels is num_panels
    
    [-2 * high_black_prop_flag[i] for i in range(num_zips)] + [1, 0], #racial aux constraint
    [2 * high_black_prop_flag[i] for i in range(num_zips)] + [1, 0],

    [-2 * high_income_flag[i] for i in range(num_zips)] + [0, 1], #equity aux constraint
    [2 * high_income_flag[i] for i in range(num_zips)] + [0, 1], #equity aux constraint
])
lb = np.array([num_panels, -num_panels, num_panels, -num_panels, num_panels])
ub = np.array([num_panels, np.inf, np.inf, np.inf, np.inf])
constraints = LinearConstraint(A, lb, ub)

# Integer constraint
integrality = np.array([1 for i in range(num_zips)] + [0, 0])  # 1 = integer, 0 = continuous

# Solve MILP
res = milp(c=total_objective, constraints=constraints, integrality=integrality, bounds=bounds)

# Output result
print("Status:", res.message)
print("Objective value:", res.fun)
print("x =", res.x)


KeyboardInterrupt: 

In [ ]:
#calculate the objectives for the generated placement
panel_placements = {zips_df['region_name'][i]: res.x[i] for i in range(num_zips)}

# projections = init_objective_projs(zip_df,objectives)

num_panels = sum(panel_placements.values()) #total panels
objectives = create_paper_objectives()

for objective in objectives:
    print(objective.name, objective.calc(zips_df, panel_placements))

Carbon Offset 25930.565671579632
Energy Potential 130733178.03265929
Racial Equity 1.500076435832268
Income Equity 1.1850516328844773


In [ ]:
import pickle
with open("test_milp_placement.pkl", "wb") as f:
    pickle.dump(panel_placements, f)